# Cypher Queries for Determining Regulatory Paths
*Núria Queralt Rosinach, Andrew Su*

**Freeze Lab meeting, January 2019**

## Overview
NGLY1 - AQP1 **regulatory review** (*NGLY1 v3.1*)

## Servers 

    * Local: bolt://kylo.scripps.edu:7689
    * AWS: bolt://52.87.232.110:7689

### Imports

In [1]:
from neo4j.v1 import GraphDatabase, basic_auth
import pandas as pd
pd.set_option('display.max_columns', None)  # or 1000
pd.set_option('display.max_rows', None)  # or 1000
pd.set_option('display.max_colwidth', -1)  # or 199

### Functions

In [2]:
def runQuery( driver, query ):
    '''
    This function runs the query onto the database and returns the result.
    in: cypher query string
    out: neo4j query result object
    '''
    
    with driver.session() as session:
        result = session.run('' + query + '')
        
    return result


def parseNode( node ):
    '''
    This function parses the information gathered in the node data structure object resulting after querying neo4j.
        in: node record neo4j object
        out: node as dict
    '''
    
    n = dict()
    n["idx"] = int(node.id)
    n["type"] = list(node.labels)[0]
    n["id"] = str(node.properties['id'])
    n["preflabel"] = str(node.properties['preflabel'])
    n["name"] = str(node.properties['name'])
    n["description"] = str(node.properties['description'])

    return n


def parsePath( path ):
    '''
    This function parsers the information gathered in the path data structure object resulting after querying neo4j.
        in: path record neo4j object
        out: path as dict
    '''
    
    out = {}
    out['Nodes'] = []
    for node in path['path'].nodes:
        n = {}
        n['idx'] = int(node.id)
        n['type'] = list(node.labels)[0]
        n['id'] = str(node.properties['id'])
        n['preflabel'] = str(node.properties['preflabel'])
        n['name'] = str(node.properties['name'])
        n['description'] = str(node.properties['description'])
        out['Nodes'].append(n)
    out['Edges'] = []
    for edge in path['path'].relationships:
        e = {}
        e['idx'] = int(edge.id)
        e['start_node'] = int(edge.start)
        e['end_node'] = int(edge.end)
        e['type'] = str(edge.type)
        e['property_label'] = str(edge.properties['property_label'])
        e['property_uri'] = str(edge.properties['property_uri'])
        e['reference_uri'] = str(edge.properties['reference_uri'])
        e['reference_date'] = str(edge.properties['reference_date'])
        e['reference_supporting_text'] = str(edge.properties['reference_supporting_text'])
        out['Edges'].append(e)
        
    return out

### Initialize neo4j

In [3]:
driver = GraphDatabase.driver("bolt://kylo.scripps.edu:7689", auth=basic_auth("neo4j", "xena"))
#driver = GraphDatabase.driver("bolt://52.87.232.110:7689")

In [ ]:
# Question
## Query topology graph
## Table of summary
## Graph of paths
## Explore paths=> executable cypher query

# TRANSCRIPTOME ANALYSIS

### Common TFs for human RNA transcriptome
* **Query web 1: < NGLY1 -[Human RNA expression (Freeze)]- gene <-- TF > order by freq**
    * ngly1-[interacts with]-gene<-tf

In [4]:
%%time
query = (
        """
        MATCH path=(source:GENE)-[i1:`RO:0002434`]-(gene:GENE)<-[i2:`RO:0002434`]-(tf:GENE)

        WHERE source.id = 'HGNC:17646' AND ALL(x IN nodes(path) WHERE single(y IN nodes(path) WHERE y = x))

        WITH path,i1,i2,tf,gene,

        [n IN nodes(path) WHERE n.preflabel IN ['cytoplasm','cytosol','nucleus','metabolism','membrane','protein binding','visible','viable','phenotype']] AS nodes_marked

        WHERE size(nodes_marked) = 0 
        
        AND toLower(i1.reference_supporting_text) =~ '.*freeze.*' 
        
        AND toLower(i2.reference_supporting_text) =~ '.*tftargets.*|.*msigdb.*'

        RETURN DISTINCT tf.id as id, tf.preflabel as symbol, tf.name as name, tf.description as description, count(distinct gene.preflabel) as freq
        
        ORDER BY freq DESC
        """
)

# run query
result = runQuery( driver, query )

# parse results
out_l = list()
for record in result:
    out_l.append({
        'TF_id': record['id'], 
        'TF_symbol': record['symbol'],
        'TF_name': record['name'],
        'TF_description': record['description'],
        'Frequency': record['freq']
    })
    
res_df = pd.DataFrame(out_l)
print(res_df.shape)

(857, 5)
CPU times: user 31 ms, sys: 1.71 ms, total: 32.8 ms
Wall time: 10.3 s


In [5]:
# Summary table
res_df[['Frequency', 'TF_id', 'TF_symbol', 'TF_name', 'TF_description']].head()

,Frequency,TF_id,TF_symbol,TF_name,TF_description
0,471,HGNC:11205,SP1,Sp1 transcription factor,"The protein encoded by this gene is a zinc finger transcription factor that binds to GC-rich motifs of many promoters. The encoded protein is involved in many cellular processes, including cell differentiation, cell growth, apoptosis, immune responses, response to DNA damage, and chromatin remodeling. Post-translational modifications such as phosphorylation, acetylation, glycosylation, and proteolytic processing significantly affect the activity of this protein, which can be an activator or a repressor. Three transcript variants encoding different isoforms have been found for this gene. [provided by RefSeq, Nov 2014]."
1,403,HGNC:6551,LEF1,lymphoid enhancer binding factor 1,"This gene encodes a transcription factor belonging to a family of proteins that share homology with the high mobility group protein-1. The protein encoded by this gene can bind to a functionally important site in the T-cell receptor-alpha enhancer, thereby conferring maximal enhancer activity. This transcription factor is involved in the Wnt signaling pathway, and it may function in hair cell differentiation and follicle morphogenesis. Mutations in this gene have been found in somatic sebaceous tumors. This gene has also been linked to other cancers, including androgen-independent prostate cancer. Alternative splicing results in multiple transcript variants. [provided by RefSeq, Oct 2009]."
2,305,HGNC:6914,MAZ,MYC associated zinc finger protein,NA
3,287,HGNC:7139,FOXO4,forkhead box O4,"This gene encodes a member of the O class of winged helix/forkhead transcription factor family. Proteins encoded by this class are regulated by factors involved in growth and differentiation indicating they play a role in these processes. A translocation involving this gene on chromosome X and the homolog of the Drosophila trithorax gene, encoding a DNA binding protein, located on chromosome 11 is associated with leukemia. Multiple transcript variants encoding different isoforms have been found for this gene. [provided by RefSeq, Jan 2010]."
4,283,NCBIGene:26196534,E12,transcription regulator,NA


### RNA gene clusters by TF and Pathway (pathways associated with gene sets)
    * Cluster RNA genes-common TF clusters by pathway (pathways associated with RNA genes in gene sets)

In [7]:
%%time
# Query 20
query = (
        """
        MATCH path=(source:GENE)-[i1:`RO:0002434`]->(g:GENE)<-[i2:`RO:0002434`]-(tf:GENE), (g:GENE)-[i]-(pw:PHYS)

        WHERE source.id = 'HGNC:17646' AND ALL(x IN nodes(path) WHERE single(y IN nodes(path) WHERE y = x))

        WITH path,i1,i2,g,tf,i,pw,

        [n IN nodes(path) WHERE n.preflabel IN ['cytoplasm','cytosol','nucleus','metabolism','membrane','protein binding','visible','viable','phenotype']] AS nodes_marked

        WHERE size(nodes_marked) = 0 
        
        AND toLower(i1.reference_supporting_text) =~ '.*freeze.*' 
        
        AND toLower(i2.reference_supporting_text) =~ '.*tftargets.*|.*msigdb.*'
        
        AND toLower(i.property_label) <> 'enables'

        RETURN DISTINCT tf.preflabel as TF_symbol, tf.name as TF_name, 
                        collect(DISTINCT g.preflabel) as geneset, count(distinct g.preflabel) as genes,
                        collect(DISTINCT pw.preflabel) as pathway, count(distinct pw.preflabel) as pathways
        
        ORDER BY genes DESC
        """
)

# run query
result = runQuery( driver, query )

# parse results
out_l = list()
for record in result:
    out_l.append({'TF symbol': record['TF_symbol'],
                  'TF name': record['TF_name'],
                  'Target geneset': record['geneset'],
                  'Total target genes': record['genes'],
                  'Pathway': record['pathway'],
                  'Total pathways': record['pathways']
                 })
    
res_df = pd.DataFrame(out_l)
print(res_df.shape)

(850, 6)
CPU times: user 166 ms, sys: 0 ns, total: 166 ms
Wall time: 39.5 s


In [8]:
# Summary table
res_df.head()

,Pathway,TF name,TF symbol,Target geneset,Total pathways,Total target genes
0,"[nucleus, regulation of transcription from RNA polymerase II promoter, plasma membrane, endoplasmic reticulum membrane, Transmembrane transport of small molecules, Metabolism, integral component of membrane, response to drug, negative regulation of apoptotic process, cellular homeostasis, endoplasmic reticulum, neuron projection, axon, integral component of plasma membrane, cellular response to retinoic acid, extracellular exosome, dendrite, glycoprotein catabolic process, Protein processing in endoplasmic reticulum, protein complex, nuclear membrane, positive regulation of cell migration, apical plasma membrane, cell-cell junction, cell surface, Post-translational protein modification, Metabolism of proteins, response to calcium ion, brush border, cellular response to oxygen-glucose deprivation, protein homooligomerization, positive regulation of transcription from RNA polymerase II promoter, Renin secretion, positive regulation of transcription, DNA-templated, transcription from RNA polymerase II promoter, protein ubiquitination, N-glycan trimming in the ER and Calnexin/Calreticulin cycle, Asparagine N-linked glycosylation, cellular response to hydrogen peroxide, cellular response to oxidative stress, positive regulation of angiogenesis, Vasopressin regulates renal water homeostasis via Aquaporins, Aquaporin-mediated transport, Vasopressin-regulated water reabsorption, renal water homeostasis, positive regulation of epithelial cell migration, caveola, erythrocyte differentiation, response to retinoic acid, basolateral plasma membrane, wound healing, protein folding, negative regulation of cysteine-type endopeptidase activity involved in apoptotic process, brush border membrane, inflammatory response, cellular response to mechanical stimulus, transmembrane transport, apical part of cell, cellular response to hypoxia, positive regulation of fibroblast proliferation, sarcolemma, O2/CO2 exchange in erythrocytes, bicarbonate transport, Erythrocytes take up carbon dioxide and release oxygen, cellular response to UV, establishment or maintenance of actin cytoskeleton polarity, anatomical structure morphogenesis, axon terminus, Proximal tubule bicarbonate reclamation, transport, odontogenesis, metanephric glomerulus vasculature development, secretion by cell, potassium ion transport, potassium ion transmembrane transport, glomerular filtration, lateral ventricle development, sensory perception of pain, cellular response to dexamethasone stimulus, cellular response to cAMP]",Sp1 transcription factor,SP1,"[RBBP8, SGK1, SERINC3, ATG5, TEX2, UNC13B, TAOK3, STMN2, ABCA1, IL6ST, STT3B, XPO1, ADAM17, TCEAL1, PHTF1, RPAP2, IGF1R, PIGK, NUCB2, TRPC1, BCL2L13, RAB22A, PTPRA, DNM1L, JKAMP, EIF2AK3, BACE2, HSPH1, CNOT7, PPP3CB, PNRC1, SDCCAG8, CALCOCO1, SRP54, TPR, ZEB2, SLC35F5, LYPD3, MAFB, RNF139, MLLT3, ARL6IP5, ATL1, PKN2, ARIH1, SETX, ZFP91, PARP8, OSBPL9, ANXA4, GPD1L, BTG1, TNFSF10, ARID4A, UBA6, XRN2, LRIG1, RAB11A, JAK2, ZNF148, SMARCA5, VPS29, FBXO11, CCND1, ADK, IGFBP2, SQOR, RAB10, APPBP2, LEPR, TNC, ELF1, CRYAB, ETFDH, GNA13, UBE4B, TRPM7, GPM6B, CROT, MGAT4B, CLDND1, PAPOLG, FAS, HK2, MOCS2, MLPH, TIMP2, ARL1, ASTN1, PCNP, CASK, PPFIA2, MITF, UST, HAT1, CIT, KDR, FEM1C, ATG3, MT2A, ...]",80,402
1,"[integral component of plasma membrane, integral component of membrane, endoplasmic reticulum, plasma membrane, neuron projection, axon, extracellular exosome, sensory perception of pain, nucleus, protein ubiquitination, Post-translational protein modification, Metabolism of proteins, Asparagine N-linked glycosylation, positive regulation of cell migration, negative regulation of apoptotic process, endoplasmic reticulum membrane, Metabolism, Protein processing in endoplasmic reticulum, transport, protein complex, positive regulation of transcription from RNA polymerase II promoter, transmembrane transport, transcription from RNA polymerase II promoter, cell-c

# NETWORK HYPOTHESES

### template that involves genes and pathways
* **Query 1: ( L = 3 ) (ngly1-ppi-pw-aqp1) ngly1-[interacts with]-gene--phys--aqp1 **
    * without degree filters
    * this query in graph v2 gives 0 paths

In [29]:
%%time
# Query 1
with driver.session() as session:
    result = session.run(
    """
    MATCH path=(source:GENE)-[:`RO:0002434`]-(:GENE)--(:PHYS)--(target:GENE)

    WHERE source.id = 'HGNC:17646' AND target.id = 'HGNC:633' AND ALL(x IN nodes(path) WHERE single(y IN nodes(path) WHERE y = x))

    WITH path,

    [n IN nodes(path) WHERE n.preflabel IN ['cytoplasm','cytosol','nucleus','metabolism','membrane','protein binding','visible','viable','phenotype']] AS nodes_marked,

    [r IN relationships(path) WHERE r.property_label IN ['in paralogy relationship with','in orthology relationship with','colocalizes with']] AS edges_marked

    WHERE size(nodes_marked) = 0 AND size(edges_marked) = 0

    RETURN count(distinct path) as paths
    """
    )

# print result
for record in result:
    print('Total paths: {}'.format(record['paths']))

Total paths: 1670
CPU times: user 1.31 ms, sys: 418 µs, total: 1.72 ms
Wall time: 553 ms


### template rna-reg L=3 min
* **Query 14: TF-pw (L=4) NGLY1--RNA_GENE--TF--PHYS--AQP1**
    * filter expression edge 
    * filter TF regulatory edge and directionality

In [9]:
%%time
# Query 14
query = (
        """
        MATCH path=(source:GENE {id: 'HGNC:17646'})-[i1:`RO:0002434`]-(rna:GENE)<-[i2:`RO:0002434`]-(tf:GENE)--(pw:PHYS)--(target:GENE {id: 'HGNC:633'})

        WHERE ALL(x IN nodes(path) WHERE single(y IN nodes(path) WHERE y = x))

        WITH path,i1,i2,rna,tf,pw,

        [n IN nodes(path) WHERE n.preflabel IN ['cytoplasm','cytosol','nucleus','metabolism','membrane','protein binding','visible','viable','phenotype']] AS nodes_marked

        WHERE size(nodes_marked) = 0 
        
        AND toLower(i1.reference_supporting_text) =~ '.*freeze.*' 
        
        AND toLower(i2.reference_supporting_text) =~ '.*tftargets.*|.*msigdb.*'

        RETURN count(distinct rna) as rnas, count(distinct tf) as tfs, count(distinct pw) as pws, count(distinct path) as paths
        
        ORDER BY paths DESC
        """
)

# run query
result = runQuery( driver, query )

# parse results
out_l = list()
for record in result:
    out_l.append({'Expressed_genes': record['rnas'], 
                  'TFs': record['tfs'],
                  'Pathways': record['pws'],
                  'Paths': record['paths']})
    
res_df = pd.DataFrame(out_l)

CPU times: user 3.98 ms, sys: 0 ns, total: 3.98 ms
Wall time: 30.4 s


In [10]:
# Summary table
res_df

,Expressed_genes,Paths,Pathways,TFs
0,1349,13467,29,221


* **Query 14.1: TF-pw (L=3) NGLY1--RNA_GENE--TF--AQP1**
    * filter expression edge 
    * filter TF regulatory edge and directionality

In [14]:
%%time
# Query 14.1
query = (
        """
        MATCH path=(source:GENE {id: 'HGNC:17646'})-[i1:`RO:0002434`]-(rna:GENE)<-[i2:`RO:0002434`]-(tf:GENE)--(target:GENE {id: 'HGNC:633'})

        WHERE ALL(x IN nodes(path) WHERE single(y IN nodes(path) WHERE y = x))

        WITH path,i1,i2,rna,tf,

        [n IN nodes(path) WHERE n.preflabel IN ['cytoplasm','cytosol','nucleus','metabolism','membrane','protein binding','visible','viable','phenotype']] AS nodes_marked

        WHERE size(nodes_marked) = 0 
        
        AND toLower(i1.reference_supporting_text) =~ '.*freeze.*' 
        
        AND toLower(i2.reference_supporting_text) =~ '.*tftargets.*|.*msigdb.*'

        RETURN count(distinct rna) as rnas, count(distinct tf) as tfs, count(distinct path) as paths
        
        ORDER BY paths DESC
        """
)

# run query
result = runQuery( driver, query )

# parse results
out_l = list()
for record in result:
    out_l.append({'Expressed_genes': record['rnas'], 
                  'TFs': record['tfs'],
                  'Paths': record['paths']})
    
res_df = pd.DataFrame(out_l)

CPU times: user 2.67 ms, sys: 0 ns, total: 2.67 ms
Wall time: 1.73 s


In [15]:
# summary table
res_df

,Expressed_genes,Paths,TFs
0,818,1839,13


### Look for a template of interest (only regulation edge): Open query + metapaths 
* **Query 13: Variable length 2-4 and pattern**
    * pattern: ngly1<-regulatory-tf-[<=3]-aqp1 
    * ngly1-[interacts with]-gene-[1..3]-aqp1 
    * filter specific regulatory types on specific edges 

In [16]:
%%time
# Query 13
query = (
        """
        MATCH path=(source:GENE)<-[i1:`RO:0002434`]-(tf:GENE)-[*..3]-(target:GENE)

        WHERE source.id = 'HGNC:17646' AND target.id = 'HGNC:633' AND ALL(x IN nodes(path) WHERE single(y IN nodes(path) WHERE y = x))

        WITH path,i1,

        [n IN nodes(path) WHERE n.preflabel IN ['cytoplasm','cytosol','nucleus','metabolism','membrane','protein binding','visible','viable','phenotype']] AS nodes_marked

        WHERE size(nodes_marked) = 0 AND toLower(i1.reference_supporting_text) =~ '.*tftargets.*|.*msigdb.*'

        RETURN count(distinct path) as paths
        """
)

# run query
result = runQuery( driver, query )

# parse results
for record in result:
    print('Paths: {}'.format(record['paths']))

Paths: 11213
CPU times: user 0 ns, sys: 3.94 ms, total: 3.94 ms
Wall time: 55.6 s


In [22]:
%%time
# Query 13
query = (
        """
        MATCH path=(source:GENE)<-[i1:`RO:0002434`]-(tf:GENE)-[*..3]-(target:GENE)

        WHERE source.id = 'HGNC:17646' AND target.id = 'HGNC:633' AND ALL(x IN nodes(path) WHERE single(y IN nodes(path) WHERE y = x))

        WITH path,i1,

        [n IN nodes(path) WHERE n.preflabel IN ['cytoplasm','cytosol','nucleus','metabolism','membrane','protein binding','visible','viable','phenotype']] AS nodes_marked

        WHERE size(nodes_marked) = 0 AND toLower(i1.reference_supporting_text) =~ '.*tftargets.*|.*msigdb.*'

        RETURN DISTINCT extract (x in rels(path) | type(x)) as types, extract (n in nodes(path) | labels(n)) as labels, length(path) as mp_length, count(distinct path) as paths 
        
        ORDER BY mp_length, paths DESC
        """
)

# run query
result = runQuery( driver, query )

# parse results
out_l = list()
for record in result:
    out_l.append({'Nodes': record['labels'], 
                  'Relations': record['types'],
                  'Metapath length': record['mp_length'],
                  'Paths': record['paths']})
    
res_df = pd.DataFrame(out_l)

CPU times: user 6.76 ms, sys: 641 µs, total: 7.4 ms
Wall time: 56 s


In [23]:
# summary table of metapaths
res_df

,Metapath length,Nodes,Paths,Relations
0,3,"[[GENE], [GENE], [GENE], [GENE]]",36,"[RO:0002434, RO:0002434, RO:0002434]"
1,3,"[[GENE], [GENE], [GENE], [GENE]]",8,"[RO:0002434, RO:HOM0000011, RO:0002434]"
2,3,"[[GENE], [GENE], [ANAT], [GENE]]",5,"[RO:0002434, RO:0002206, RO:0002206]"
3,4,"[[GENE], [GENE], [GENE], [GENE], [GENE]]",6534,"[RO:0002434, RO:0002434, RO:0002434, RO:0002434]"
4,4,"[[GENE], [GENE], [GENE], [ANAT], [GENE]]",1502,"[RO:0002434, RO:0002434, RO:0002206, RO:0002206]"
5,4,"[[GENE], [GENE], [GENE], [PHYS], [GENE]]",432,"[RO:0002434, RO:0002434, BFO:0000050, BFO:0000050]"
6,4,"[[GENE], [GENE], [GENE], [PHYS], [GENE]]",425,"[RO:0002434, RO:0002434, RO:0002331, RO:0002331]"
7,4,"[[GENE], [GENE], [GENE], [GENE], [GENE]]",400,"[RO:0002434, RO:HOM0000011, RO:0002434, RO:0002434]"
8,4,"[[GENE], [GENE], [GENE], [GENE], [GENE]]",208,"[RO:0002434, RO:HOM0000011, RO:HOM0000011, RO:0002434]"
9,4,"[[GENE], [GENE], [GENE], [GENE], [GENE]]",190,"[RO:0002434, RO:0002325, RO:0002434, RO:0002434]"


### template refinement (only regulation edge)
* **Query 12: ( L = 4 ) ngly1--TF-[regulatory]->gene--phys--aqp1 **
    * filter specific regulatory types on specific edges 
    * filter directionality
    * filter node type on third node, otherwise runtime is long

In [24]:
%%time
# Query 12
query = (
        """
        MATCH path=(source:GENE)--(tf:GENE)-[i1:`RO:0002434`]->(:GENE)--(:PHYS)--(target:GENE)

        WHERE source.id = 'HGNC:17646' AND target.id = 'HGNC:633' AND ALL(x IN nodes(path) WHERE single(y IN nodes(path) WHERE y = x))

        WITH path,i1,

        [n IN nodes(path) WHERE n.preflabel IN ['cytoplasm','cytosol','nucleus','metabolism','membrane','protein binding','visible','viable','phenotype']] AS nodes_marked

        WHERE size(nodes_marked) = 0 AND toLower(i1.reference_supporting_text) =~ '.*tftargets.*|.*msigdb.*'

        RETURN count(distinct path) as paths
        """
)

# run query
result = runQuery( driver, query )

# parse results
for record in result:
    print('Paths: {}'.format(record['paths']))

Paths: 15628
CPU times: user 3.34 ms, sys: 0 ns, total: 3.34 ms
Wall time: 29.5 s


* **Query 11: ( L = 3 ) ngly1--TF-[regulatory]->gene--aqp1 **
    * filter specific regulatory types on specific edges 
    * filter directionality

In [25]:
%%time
# Query 11
query = (
        """
        MATCH path=(source:GENE)--(tf:GENE)-[i1:`RO:0002434`]->(:GENE)--(target:GENE)

        WHERE source.id = 'HGNC:17646' AND target.id = 'HGNC:633' AND ALL(x IN nodes(path) WHERE single(y IN nodes(path) WHERE y = x))

        WITH path,i1,

        [n IN nodes(path) WHERE n.preflabel IN ['cytoplasm','cytosol','nucleus','metabolism','membrane','protein binding','visible','viable','phenotype']] AS nodes_marked

        WHERE size(nodes_marked) = 0 AND toLower(i1.reference_supporting_text) =~ '.*tftargets.*|.*msigdb.*'

        RETURN count(distinct path) as paths
        """
)

# run query
result = runQuery( driver, query )

# parse results
for record in result:
    print('Paths: {}'.format(record['paths']))

Paths: 338
CPU times: user 1.94 ms, sys: 464 µs, total: 2.4 ms
Wall time: 1.22 s


## Mitali queries
### 14 Jan 2019 meeting 

Here are the details of the querry we talked about yesterday;


Look for gene-gene interactions linking NGLY1 and AQPs through the following genes;

* ATF1 (ENSG00000123268)
    * https://uswest.ensembl.org/Homo_sapiens/Gene/Summary?g=ENSG00000123268;r=12:50763710-50821122

* CREB1 (ENSG00000118260)
* PRKACA (ENSG00000072062)
* cyclic adenosine monophosphate (cAMP)
* **cAMP** is not a protein so there is no identification number for it. 

_Note_: Protein kinase A has multiple subunits. Each subunit has a different identifier. I have provided you with the gene identifier for its catalytic subunit PRKACA.


**QUERY TOPOLOGY**:

*  NGLY1-interacts-(gene)-interacts-(cAMP|ATF1|CREB1|PKA),  with length n=1/2

* **Query: ( L = 3 ) ngly1--TF-[regulatory]->gene--atf1 **
    * filter specific regulatory types on specific edges 
    * filter directionality

In [6]:
%%time
query = (
        """
        MATCH path=(source:GENE)-[i1:`RO:0002434`]-(tf:GENE)-[*..2]-(target:GENE)

        WHERE source.id = 'HGNC:17646' AND target.id = 'HGNC:783' AND ALL(x IN nodes(path) WHERE single(y IN nodes(path) WHERE y = x))

        WITH path,i1,

        [n IN nodes(path) WHERE n.preflabel IN ['cytoplasm','cytosol','nucleus','metabolism','membrane','protein binding','visible','viable','phenotype']] AS nodes_marked

        WHERE size(nodes_marked) = 0 AND toLower(i1.reference_supporting_text) =~ '.*tftargets.*|.*msigdb.*'

        RETURN count(distinct path) as paths
        """
)

# run query
result = runQuery( driver, query )

# parse results
for record in result:
    print('Paths: {}'.format(record['paths']))



Paths: 141
CPU times: user 2.26 ms, sys: 255 µs, total: 2.52 ms
Wall time: 1.79 s


In [10]:
%%time
query = (
        """
        MATCH path=(source:GENE)--(target:GENE)

        WHERE source.id = 'HGNC:17646' AND target.id = 'HGNC:783' AND ALL(x IN nodes(path) WHERE single(y IN nodes(path) WHERE y = x))

        WITH path,

        [n IN nodes(path) WHERE n.preflabel IN ['cytoplasm','cytosol','nucleus','metabolism','membrane','protein binding','visible','viable','phenotype']] AS nodes_marked

        WHERE size(nodes_marked) = 0 

        RETURN count(distinct path) as paths
        """
)

# run query
result = runQuery( driver, query )

# parse results
for record in result:
    print('Paths: {}'.format(record['paths']))

Paths: 0
CPU times: user 1.64 ms, sys: 182 µs, total: 1.82 ms
Wall time: 45.3 ms


In [7]:
%%time
query = (
        """
        MATCH path=(source:GENE)-[i1:`RO:0002434`]-(target:GENE)

        WHERE source.id = 'HGNC:17646' AND target.id = 'HGNC:783' AND ALL(x IN nodes(path) WHERE single(y IN nodes(path) WHERE y = x))

        WITH path,

        [n IN nodes(path) WHERE n.preflabel IN ['cytoplasm','cytosol','nucleus','metabolism','membrane','protein binding','visible','viable','phenotype']] AS nodes_marked

        WHERE size(nodes_marked) = 0 

        RETURN count(distinct path) as paths
        """
)

# run query
result = runQuery( driver, query )

# parse results
for record in result:
    print('Paths: {}'.format(record['paths']))

Paths: 0
CPU times: user 1.89 ms, sys: 210 µs, total: 2.1 ms
Wall time: 62 ms


In [8]:
%%time
query = (
        """
        MATCH path=(source:GENE)-[i1:`RO:0002434`]-(:GENE)-[i2:`RO:0002434`]-(target:GENE)

        WHERE source.id = 'HGNC:17646' AND target.id = 'HGNC:783' AND ALL(x IN nodes(path) WHERE single(y IN nodes(path) WHERE y = x))

        WITH path,

        [n IN nodes(path) WHERE n.preflabel IN ['cytoplasm','cytosol','nucleus','metabolism','membrane','protein binding','visible','viable','phenotype']] AS nodes_marked

        WHERE size(nodes_marked) = 0 

        RETURN count(distinct path) as paths
        """
)

# run query
result = runQuery( driver, query )

# parse results
for record in result:
    print('Paths: {}'.format(record['paths']))

Paths: 92
CPU times: user 1.9 ms, sys: 0 ns, total: 1.9 ms
Wall time: 78.3 ms


In [9]:
%%time
query = (
        """
        MATCH path=(source:GENE)-[i1:`RO:0002434`]-(:GENE)-[i2:`RO:0002434`]-(:GENE)-[i3:`RO:0002434`]-(target:GENE)

        WHERE source.id = 'HGNC:17646' AND target.id = 'HGNC:783' AND ALL(x IN nodes(path) WHERE single(y IN nodes(path) WHERE y = x))

        WITH path,i1,

        [n IN nodes(path) WHERE n.preflabel IN ['cytoplasm','cytosol','nucleus','metabolism','membrane','protein binding','visible','viable','phenotype']] AS nodes_marked

        WHERE size(nodes_marked) = 0 

        RETURN count(distinct path) as paths
        """
)

# run query
result = runQuery( driver, query )

# parse results
for record in result:
    print('Paths: {}'.format(record['paths']))

Paths: 12042
CPU times: user 1.89 ms, sys: 0 ns, total: 1.89 ms
Wall time: 2.95 s
